In [1]:
import json
import math
import gurobipy as gp
from gurobipy import Model, GRB, quicksum



In [2]:
problem = 'tests/drone_problem_1.json'
with open(problem, 'r') as file:
    data = json.load(file)

# Extract data from JSON
drone_weight = data['drone_weight']
max_capacity = data['max_capacity']
battery_capacity = data['battery_capacity']
points = data['pontos']

points


[{'x': 55, 'y': 65, 'peso': 0},
 {'x': 95, 'y': 93, 'peso': 5},
 {'x': 96, 'y': 79, 'peso': 10},
 {'x': 92, 'y': 67, 'peso': 8},
 {'x': 89, 'y': 74, 'peso': 4},
 {'x': 29, 'y': 44, 'peso': 10},
 {'x': 55, 'y': 65, 'peso': 0}]

In [3]:
n = len(points) - 2  
base_start = 0
base_end = n + 1


d = {}
for i in range(len(points)):
    for j in range(len(points)):
        if i != j:
            d[i, j] = math.sqrt((points[i]['x'] - points[j]['x'])**2 + (points[i]['y'] - points[j]['y'])**2)
        else:
            d[i, j] = 0


p = {}
for i in range(1, len(points) - 1):
    p[i] = points[i]['peso']


p[base_start] = 0
p[base_end] = 0


In [29]:
def my_callback(model, where):
    if where == GRB.Callback.MIPSOL:  # Triggered when a new solution is found
        # Get the time since the start of optimization
        solution_time = model.cbGet(GRB.Callback.RUNTIME)
        # Retrieve the objective value of the solution
        obj_val = model.cbGet(GRB.Callback.MIPSOL_OBJ)
        
        # Store the results
        results.append({'time': solution_time, 'objective_value': obj_val})

env = gp.Env(empty=True)
env.setParam('LICENSEID', 2548345)
env.start()
model=Model('drone')
model.setParam('TimeLimit', 500)
alpha = 1
beta = 1
x = model.addVars(len(points), len(points), vtype=GRB.BINARY, name="x")
omega = model.addVars(len(points), vtype=GRB.CONTINUOUS, name="omega")
e = model.addVars(len(points), len(points), vtype=GRB.CONTINUOUS, name="e")


#1. cada casa so vai para 1 lugar
for i in range(1, n + 1):
    model.addConstr(quicksum(x[i, j] for j in range(1, n + 2)) == 1, name=f"c1_{i}")
#2.nao vai da base para a base
model.addConstr(x[base_start, base_end] == 0, name="c2")
#3. apos voltar, nao sai
for j in range(len(points)):
    model.addConstr(x[base_end, j] == 0, name=f"c3_{j}")
#4. nao vai da casa para ela mesma
for i in range(len(points)):
    model.addConstr(x[i, i] == 0, name=f"c4_{i}")
#5. nao vai para base inicial
for i in range(len(points)):
    model.addConstr(x[i, base_start] == 0, name=f"c5_{i}")
#6. entra na base o mesmo numero de vezes que sai
model.addConstr(quicksum(x[i, base_end] for i in range(len(points))) == quicksum(x[base_start, j] for j in range(len(points))), name="c6")
#7. cada casa eh visitada 1 vez
for j in range(1, n + 1):
    model.addConstr(quicksum(x[i,j] for i in range(len(points))) == 1, name=f"c7_{j}")
#8. nenhum omega é maior que a capacidade
for i in range(len(points)):
    model.addConstr(omega[i] <= max_capacity, name=f"c8_{i}")
#9. omega da base final é o peso do drone
model.addConstr(omega[base_end] == drone_weight, name="c9")
# 10. Calculo de omega
for i in range(1, n+1):
    model.addConstr(omega[i] == p[i] + quicksum((omega[j])*x[i,j] for j in range(len(points))), name=f"c10_{i}")
# 11. Calculo de e inicial
for j in range(len(points)):
    model.addConstr(e[base_start, j] == battery_capacity - (((d[base_start, j])) * ((omega[j]) * x[base_start, j])), name=f"c11_{j}")
#12. Calculo de e
for i in range(1, n+2):
    for j in range(len(points)):
        model.addConstr(e[i, j] == quicksum(e[k,i]*x[k,i] for k in range(len(points))) - (((d[i, j])) * ((omega[j]) * x[i, j])), name=f"c12_{i}_{j}")
#13. e deve ser maior ou igual a 0
for i in range(len(points)):
    for j in range(len(points)):
        model.addConstr(e[i, j] >= 0, name=f"c13_{i}_{j}")

model.setObjective(quicksum(x[i,j] * (d[i,j]) * omega[j] for i in range(len(points)) for j in range(len(points))), GRB.MINIMIZE)

results = []

model.optimize(my_callback)










Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2548345
Academic license 2548345 - for non-commercial use only - registered to ra___@aluno.cefetmg.br
Set parameter TimeLimit to value 500
Set parameter WLSSecret
Set parameter LicenseID to value 2548345
Academic license 2548345 - for non-commercial use only - registered to ra___@aluno.cefetmg.br
Set parameter TimeLimit to value 500
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD Ryzen 7 5800X 8-Core Processor, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Academic license 2548345 - for non-commercial use only - registered to ra___@aluno.cefetmg.br
Optimize a model with 90 rows, 105 columns and 156 nonzeros
Model fingerprint: 0x406a871a
Model has 40 quadratic objective terms
Model has 54 quadratic constraints
Variable types: 56 continuous, 49 integer (49 binary)
Coefficient statistics:
  Mat

In [5]:
for var in model.getVars():
    if(var.x > 0):
        print(f"{var.varName} = {var.x}")


#print solution objective value
print(f"Objetivo: {model.objVal}")

x[0,2] = 1.0
x[0,3] = 1.0
x[0,5] = 1.0
x[1,6] = 1.0
x[2,1] = 1.0
x[3,4] = 1.0
x[4,6] = 1.0
x[5,6] = 1.0
omega[1] = 15.0
omega[2] = 25.0
omega[3] = 22.0
omega[4] = 14.0
omega[5] = 20.0
omega[6] = 10.0
e[0,0] = 2200.0
e[0,1] = 2200.0
e[0,2] = 1116.8910488782767
e[0,3] = 1384.8116781994372
e[0,4] = 2200.0
e[0,5] = 1531.5690013172618
e[0,6] = 2200.0
e[1,0] = 906.3560161639998
e[1,1] = 906.3560161639998
e[1,2] = 906.3560161640019
e[1,3] = 906.3560161640019
e[1,4] = 906.3560161640006
e[1,5] = 906.356016163997
e[1,6] = 418.0937915346517
e[2,0] = 1116.8910488782774
e[2,1] = 906.3560161640025
e[2,2] = 1116.8910488782774
e[2,3] = 1116.8910488782778
e[2,4] = 1116.8910488782763
e[2,5] = 1116.8910488782778
e[2,6] = 1116.8910488782774
e[3,0] = 1384.8116781994352
e[3,1] = 1384.811678199435
e[3,2] = 1384.8116781994363
e[3,3] = 1384.8116781994352
e[3,4] = 1278.1908547173384
e[3,5] = 1384.8116781994363
e[3,6] = 1384.8116781994352
e[4,0] = 1278.190854717338
e[4,1] = 1278.1908547173377
e[4,2] = 1278.19085

In [6]:
subroutes = []
goesTo = {}

for var in model.getVars():
    if var.VarName[0] == 'x' and var.x > 0:
        #print(f"{var.VarName} = {var.x}")
        x,y = var.VarName.split('[')[1].split(']')[0].split(',')
        x = int(x)
        y = int(y)
        if x == base_start:
            subroutes.append([y])
        else:
            goesTo[x] = y

for i in range(len(subroutes)):
    next = goesTo[subroutes[i][-1]]
    while next != base_end:
        subroutes[i].append(next)
        next = goesTo[next]

print(subroutes)
route = []
for subroute in subroutes:
    route += subroute
    route.append(0)
route.pop()
print(route)
print(model.ObjVal)
        

[[2, 1], [3, 4], [5]]
[2, 1, 0, 3, 4, 0, 5]
4058.071959673475


In [28]:
solution_count = model.SolCount
solve_json = {}
problem_name = problem.split('/')[1].split('.')[0]
with open(f"results/solved_{problem_name}.json", 'w') as file:
    json.dump(results, file)